In [ ]:
# 사전에 훈련된 모델 로드
model_grad_cam = transfer_model_best
model_grad_cam.to(device)
model_grad_cam.eval()

# 마지막 합성곱 층의 특성 맵과 그래디언트를 저장할 변수
feature_maps = None
gradients = None

# Hook 함수
def get_features_hook(module, input, output):
    global feature_maps
    feature_maps = output

def get_gradients_hook(module, input_grad, output_grad):
    global gradients
    gradients = output_grad[0]

# 마지막 합성곱 층에 Hook 등록
target_layer = model_grad_cam.layer4[1].conv2
target_layer.register_forward_hook(get_features_hook)
target_layer.register_full_backward_hook(get_gradients_hook)

# Grad-CAM 계산 함수
def compute_grad_cam(input_image, class_idx):
    global feature_maps, gradients
    input_image = input_image.to(device)
    # 모델의 예측
    output = model_grad_cam(input_image)
    
    # 클래스 확률 최대화를 위한 그래디언트
    model_grad_cam.zero_grad()
    class_loss = output[0, class_idx]
    class_loss.backward()
    
    # 그래디언트 가중 평균 계산
    pooled_gradients = torch.mean(gradients, dim=[0, 2, 3])
    
    # 특성 맵에 그래디언트를 곱한 후 평균을 냄
    for i in range(pooled_gradients.size(0)):
        feature_maps[0, i, :, :] *= pooled_gradients[i]
    
    # Grad-CAM
    grad_cam = torch.mean(feature_maps, dim=1).squeeze().cpu()
    grad_cam = F.relu(grad_cam)
    
    # Grad-CAM 리사이징
    grad_cam_resized = F.interpolate(grad_cam.unsqueeze(0).unsqueeze(0), 
                                     size=input_image.shape[2:], 
                                     mode='bilinear', 
                                     align_corners=False).squeeze()
    
    grad_cam_resized = grad_cam_resized / grad_cam_resized.max()
    
    return grad_cam_resized.detach().numpy()

# 데이터 로더에서 이미지 가져오기
for images, labels in test_loader:
    images = images.to(device)

    # 이미지 전처리 및 디바이스 설정
    plt.figure(figsize=(12,3))
    for idx, image in enumerate(images):
        original_image = tensor_to_img(image)
        input_image = image.unsqueeze(0)  # 첫 번째 이미지
        input_image.requires_grad = True

        # 클래스 인덱스 (예: 고양이가 클래스 0)
        class_idx = labels[idx].item()

        # Grad-CAM 계산
        grad_cam = compute_grad_cam(input_image, class_idx)
        
        plt.subplot(1,len(images),idx+1)

        # 이미지와 Grad-CAM 시각화
        plt.imshow(original_image)
        plt.imshow(grad_cam, cmap='magma', alpha=0.5)
        plt.axis('off')
    plt.show()